## Skorch is a scikit-learn compatible neural network library that wraps PyTorch. This allows performing manipulations with PyTorch modules as if they were usual scikit-learn estimators. In particular, I find it very useful to do a grid search for PyTorch models via Skorch. 
## In this notebook, I show how one can easily do a grid search for PyTorch model wrapped in Skorch with pre-trained weights initialization at each fit. 
### Although it is not recommended to use pre-defined weights initialization for general purposes, it might be useful for, e.g., exhaustive fine tuning.


### First, let's import libraries and download the MNIST dataset,

In [83]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import skorch
from skorch import NeuralNet
from skorch import NeuralNetClassifier

import pickle

import numpy as np

### The dataset is not that large so let's store it in memory.

In [85]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: ',device)


# Download MNIST dataset to local drive. A new folder "data" will be created in the current directory to store data
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=len(train_dataset), 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=len(test_dataset), 
                                          shuffle=False)

# Train set
for X, y in train_loader:
    X_train = X
    y_train = y

# Test set
for X, y in test_loader:
    X_test = X
    y_test = y

Device:  cuda


### Let's build a core NN module with PyTorch
#### Two hidden layers. ReLU activation in hidden layers and logsoftmax activation in output layer. Let's also use dropout.

In [38]:
class MyModule(nn.Module):

    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes, dropout):
        super(MyModule, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size1) 
        self.relu =nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2) 
        self.fc3 = nn.Linear(hidden_size2, num_classes)  
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):

        x = x.view(-1, 784)
        out = self.dropout(self.fc1(x))
        out = self.relu(out)
        out = self.dropout(self.fc2(out))
        out = self.relu(out)
        out = self.fc3(out)
        out = self.logsoftmax(out)
                        
        return out

### Here we inherit NeuralNetClassifier class from Skorch but with two modifications.
#### 1) We slightly modify get_loss to add L1 regularization.
#### 2) We modify initialize_module so we can use pre-trained initialization in GridSearchCV if needed.
####    *reinit_from_pretrain*   indicates if the model should be initialized from a pre-trained one.
####    *pretrained_nn*   contains a path to a pickle with the pre-trained model.

In [97]:
class RegularizedNet(NeuralNetClassifier):
    
    def __init__(self, *args, lambda1=0.01, reinit_from_pretrain=False, pretrained_nn=None, **kwargs):
        '''
        If reinit_from_pretrain = True, the model is initialized from a pretrained pickle.
        pretrained_nn containt path to the pickle.
        Otherwise, the model is randomly re-initialized at each fit.
        '''
        super().__init__(*args, **kwargs)
        self.lambda1 = lambda1
        self.reinit_from_pretrain = reinit_from_pretrain
        self.pretrained_nn = pretrained_nn
        
    def get_loss(self, y_pred, y_true, X=None, training=False):
        loss = super().get_loss(y_pred, y_true, X=X, training=training)
        loss += self.lambda1 * sum([w.abs().sum() for w in self.module_.parameters()])
        return loss
    
    def initialize_module(self):
        
        super().initialize_module()
        
        ##### initialization from pre-trained model
        if self.reinit_from_pretrain:
            self.module_.load_state_dict(torch.load(self.pretrained_nn))
        return self

#### Here we wrap Skorch around the PyTorch module.

In [88]:
##### Below I define collbacks for train accuracy printing and early stopping criteria.
train_acc = skorch.callbacks.EpochScoring(scoring = accuracy_score,on_train=True, 
                         name='train_acc', lower_is_better=False)

early_stop = skorch.callbacks.EarlyStopping(monitor='valid_acc', lower_is_better=False)

callbacks = [train_acc, early_stop]


new_net = RegularizedNet(module=MyModule, criterion=torch.nn.CrossEntropyLoss, device='cuda',
                        optimizer=torch.optim.SGD, lr = 0.2, lambda1 = 0,  module__dropout = 0.2,
                        optimizer__weight_decay = 0.0, max_epochs = 45, callbacks=callbacks, batch_size=256,
                        module__input_size= 784, module__hidden_size1=128, module__hidden_size2=64, module__num_classes=10)

#### Let's train the NN.

In [89]:
new_net.fit(X_train, y = y_train)
y_pred_probs = new_net.predict(test_dataset)

  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.7355        0.9199       0.8952        0.3593  0.6321
      2       0.8920        0.3687       0.9203        0.2652  0.6481
      3       0.9165        0.2868       0.9334        0.2170  0.6802
      4       0.9305        0.2378       0.9419        0.1878  0.6561
      5       0.9399        0.2070       0.9538        0.1579  0.6371
      6       0.9460        0.1831       0.9585        0.1391  0.6752
      7       0.9522        0.1634       0.9610        0.1282  0.6351
      8       0.9551        0.1492       0.9648        0.1192  0.6792
      9       0.9595        0.1386       0.9646        0.1149  0.6391
     10       0.9614        0.1306       0.9686        0.1033  0.6371
     11       0.9629        0.1238       0.9674        0.1059  0.6772
     12       0.9661        0.1142       0.9700        0.0984  0.6381
     13       0.9677

#### The test accuracy is 97.95%. We will use this pre-trained model for weights initialization during GridSearchCV.

In [100]:
y_pred = new_net.predict(X_test)
accuracy_score(y_test.cpu(),  torch.tensor(y_pred))*100

97.95

#### Save the model

In [92]:
new_net.save_params(f_params='pre-trained-NN.pkl')

### Now, let's make a grid search with sklearn keeping weights initialization from a pre-trained model at each fit.
### Keep in mind that we cannot alter the number of neurons in each layer anymore because we initialize the model from a pre-trained one.

In [93]:
new_net_gs = RegularizedNet(module=MyModule, criterion=torch.nn.CrossEntropyLoss, device='cuda',
                        optimizer= torch.optim.SGD, lr = 0.2, lambda1 = 0.0,  module__dropout = 0.0,
                        optimizer__weight_decay = 0, max_epochs = 45,callbacks=callbacks, batch_size=256,
                        module__input_size= 784, module__hidden_size1=128, module__hidden_size2=64, module__num_classes=10,
                        reinit_from_pretrain=True, pretrained_nn='pre-trained-NN.pkl')


grid = {
    'lambda1': [ 0.0, 0.0001],
    'lr': [0.01, 0.05],
    }


gs = GridSearchCV(new_net_gs, grid, refit=True, cv=3, verbose=3)

gs.fit(X_train, y_train)

#Report Best Parameters
print(gs.best_score_, gs.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9961        0.0152       0.9971        0.0150  0.4121
      2       0.9965        0.0141       0.9975        0.0148  0.4521
      3       0.9967        0.0135       0.9975        0.0146  0.4121
      4       0.9968        0.0130       0.9975        0.0145  0.4191
      5       0.9970        0.0127       0.9975        0.0144  0.4091
      6       0.9972        0.0124       0.9976        0.0143  0.4521
      7       0.9974        0.0121       0.9976        0.0142  0.4111
      8       0.9975        0.0119       0.9975        0.0141  0.4121
      9       0.9977        0.0117       0.9975        0.0141  0.4111
     10       0.9978        0.0115       0.9975        0.0141  0.4591
Stopping since valid_acc has not improved in the last 5 epochs.
[CV 1/3] END .......................

      3       0.9950        0.4314       0.9791        0.4770  0.4551
      4       0.9952        0.4295       0.9789        0.4756  0.4151
      5       0.9954        0.4278       0.9792        0.4742  0.4121
      6       0.9956        0.4261       0.9791        0.4728  0.4131
Stopping since valid_acc has not improved in the last 5 epochs.
[CV 2/3] END ........................lambda1=0.0001, lr=0.01; total time=   3.2s
  epoch    train_acc    train_loss    valid_acc    valid_loss     dur
-------  -----------  ------------  -----------  ------------  ------
      1       0.9940        0.4371       0.9792        0.4806  0.4261
      2       0.9943        0.4345       0.9792        0.4787  0.4161
      3       0.9947        0.4323       0.9791        0.4771  0.4601
      4       0.9949        0.4304       0.9789        0.4757  0.4131
      5       0.9948        0.4286       0.9788        0.4742  0.4121
Stopping since valid_acc has not improved in the last 5 epochs.
[CV 3/3] END ........

### Final test accuracy.

In [99]:
y_pred = gs.best_estimator_.predict(X_test)
accuracy_score(y_test.cpu(),  torch.tensor(y_pred))*100

98.03

In [82]:
#gs.cv_results_